In [1]:
%matplotlib inline

# Linear Models and PyMC3

So far we have covered the tools we need for basic parameter estimation when we have a single unknown parameter. There is a lot you can do with just this information, but the real work horse of statistical modeling is linear models.

In [2]:
from context import src
from src import customer as cust
from src import product as prod
from src import experiment as exp
from src import messy_experiment as messy_exp

import pymc3 as pm
import arviz as az

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.special import expit as logistic

## Revisiting the Toothbrushes as a linear model

In [3]:
toothbrush = prod.Product(name="alright brush",
                          price=4.99,
                          quality=3.9)

luxury_toothbrush = prod.Product(name="luxury toothbrush",
                                 price=7.99,
                                 quality=4.8)

toothbrush_ab_test = exp.Experiment(toothbrush,luxury_toothbrush)
n_samples = 100
experiment_results = toothbrush_ab_test.show_to_customers(n_samples)

In [4]:
purchased = np.concatenate((experiment_results.a_purchased.astype(int),
                            experiment_results.b_purchased.astype(int)))
is_luxury = np.concatenate((np.zeros(n_samples),
                            np.ones(n_samples)))
ab_test_data = pd.DataFrame({
    'purchased' : purchased,
    'is_luxury' : is_luxury
})

In [ ]:
with pm.Model() as logistic_model:
    pm.glm.GLM.from_formula('purchased ~ 1 + is_luxury',
                            ab_test_data,
                            family=pm.glm.families.Binomial())
    ab_trace = pm.sample(1000, tune=1000, init='adapt_diag')

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...


In [ ]:
pm.traceplot(ab_trace)

Discuss what this mean in the model...

In [ ]:
lo_is_luxury = ab_trace.get_values('is_luxury')

print(sum(lo_is_luxury > 0)/len(lo_is_luxury))

In [ ]:
lo_regular = ab_trace.get_values('Intercept')

In [ ]:
sns.distplot(logistic(lo_regular),hist=False)
sns.distplot(logistic(lo_is_luxury + lo_regular),hist=False)

Compare this with just using our Beta distribution estimates...

In [ ]:
alpha_a = sum(experiment_results.a_purchased)
beta_a = n_samples - alpha_a 
a_beta_dist = stats.beta(alpha_a,beta_a)

alpha_b = sum(experiment_results.b_purchased)
beta_b = n_samples - alpha_b 
b_beta_dist = stats.beta(alpha_b,beta_b)
rates = np.arange(0,0.3,0.001)
plot_df = pd.DataFrame({
    'density':np.concatenate((a_beta_dist.pdf(rates),
                              b_beta_dist.pdf(rates))),
    'rate': np.concatenate((rates,rates)),
    'group':['regular']*len(rates) + ['luxury']*len(rates)
})
sns.lineplot(x='rate',
             y='density',
             hue='group',
             data=plot_df)

nearly the same result!

but why would we care? 

Using a linear model gives us the same result, but also makes it much easier for us to extend our solution to more complicated issues.

## More complicated issues!

Now let's suppose we have a situation where we have two other products that we want to sell:

In [ ]:
very_nice_pencil = prod.Product(name="Very nice pencil",
                                price=4.99,
                                quality=4.9)
deluxe_pencil = prod.Product(name="Deluxe pencil",
                             price=4.25,
                             quality=4.8)

We're going to run another experiment testing this values, but in this case we'll simulate when we don't design our experiment quite right:

In [ ]:
pencil_test = messy_exp.MessyExperiment(very_nice_pencil,
                                        deluxe_pencil)

We'll run this test for a bunch of people so we can get some pretty good results

In [ ]:
n_samples = 2500
test_results = pencil_test.show_to_customers(n_samples)

In [ ]:
sum(test_results.a_purchased)/n_samples

In [ ]:
sum(test_results.b_purchased)/n_samples

In [ ]:
purchased = np.concatenate((test_results.a_purchased.astype(int),
                            test_results.b_purchased.astype(int)))
is_deluxe = np.concatenate((np.zeros(n_samples),
                            np.ones(n_samples)))
pencil_test_data = pd.DataFrame({
    'purchased' : purchased,
    'is_deluxe' : is_deluxe
})

with pm.Model() as pencil_test_model:
    pm.glm.GLM.from_formula('purchased ~ 1 + is_deluxe',
                            pencil_test_data,
                            family=pm.glm.families.Binomial())
    pencil_test_trace = pm.sample(1000, tune=1000, init='adapt_diag')

In [ ]:
pm.traceplot(pencil_test_trace)

In [ ]:
is_deluxe_samples = pencil_test_trace.get_values("is_deluxe")
sum(is_deluxe_samples < 0)/len(is_deluxe_samples)

Both of these pencils sold at a pretty low rate, which isn't surprsiing because they're pretty expensive. What is surprising is that it look like the `deluxe_pencil` sold for a much lower rate than the `very_nice_pencil`, even though both products are pretty much the same.

If we take a peak at our data we can get an idea of what's happening:

In [ ]:
test_results.a_customer.head(10)

In [ ]:
test_results.b_customer.head(10)

There were students in these experiments! And if we look at the two experiment groups we can see that the count is different:

In [ ]:
sum(test_results.a_customer == "student")

In [ ]:
sum(test_results.b_customer == "student")

The b group go exposed to much more students than the a group! Could this explain what happened? It's reasonable to assume that students might have a lower threshold for the price they are willing to purchase a pen for

In [ ]:
pencil_test_data['is_student'] = np.concatenate(
    (test_results.a_customer ==  "student",
     test_results.b_customer == "student")).astype(int)

In [ ]:
pencil_test_data[pencil_test_data.is_student == 1].purchased.sum()

In [ ]:
with pm.Model() as pencil_test_student_model:
    pm.glm.GLM.from_formula('purchased ~ 1 + is_deluxe + is_student',
                            pencil_test_data,
                            family=pm.glm.families.Binomial())
    pencil_test_student_trace = pm.sample(1000, tune=1000, init='adapt_diag')

In [ ]:
pm.traceplot(pencil_test_student_trace)